# Modelos para probabilidad de respuesta

Importamos librerías:

In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)

In [2]:
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
      QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [24]:
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

Cambiamos directorio de trabajo:

In [3]:
os.chdir('C:/Users/celin/OneDrive/Documentos/CELINE/FACULTAD/Maestría en Estadística Aplicada/4to semestre/Tesis/Nonresponse-bias-eph')
print(os.getcwd())

C:\Users\celin\OneDrive\Documentos\CELINE\FACULTAD\Maestría en Estadística Aplicada\4to semestre\Tesis\Nonresponse-bias-eph


Cargamos la base de datos individual y hogar:

In [7]:
individual_NEA = pd.read_table("Bases/individual_NEA_prediccion.txt", delimiter = ";", low_memory=False)
individual_NEA.head()

,periodo,AGLOMERADO,ANO4,TRIMESTRE,CODUSU,NRO_HOGAR,REGION_DESC,REGION_COD,CH04,CH06,...,mujer,otros_ing_nolab,anio,casadounido,casadpto,leer,basural,n_entrevista,area,completo
0,2023.25,15,2023,2,TQRMNOPPQHJMKNCDEGLDF00805128,1,Noreste,41,2,73,...,1,58000.0,2023,0,1,1,0,1,TQRMNOPP,0
1,2017.25,8,2017,2,TQRMNOPPQHJMKRCDEFOCD00472238,1,Noreste,41,1,62,...,0,0.0,2017,1,1,1,0,3,TQRMNOPP,1
2,2017.50,8,2017,3,TQRMNOPPQHJMKRCDEFOCD00472238,1,Noreste,41,1,62,...,0,0.0,2017,1,1,1,0,4,TQRMNOPP,1
3,2020.75,7,2020,4,TQRMNOPPQHJMLPCDEFNFF00670213,1,Noreste,41,2,52,...,1,0.0,2020,0,1,1,0,1,TQRMNOPP,1
4,2021.00,7,2021,1,TQRMNOPPQHJMLPCDEFNFF00670213,1,Noreste,41,2,52,...,1,0.0,2021,0,1,1,0,2,TQRMNOPP,1


In [9]:
individual_NEA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34860 entries, 0 to 34859
Columns: 223 entries, periodo to completo
dtypes: float64(126), int64(78), object(19)
memory usage: 59.3+ MB


## Selección de variables, stepwise.

## Modelo de regresión logístico

Dividimos la muestra en entrenamiento y test:

In [11]:
individual_NEA_train, individual_NEA_test = train_test_split(individual_NEA, test_size=0.30, random_state=0)

In [19]:
individual_NEA_train.shape 

(24402, 223)

In [20]:
individual_NEA_test.shape

(10458, 223)

In [49]:
allvars = individual_NEA[['logIPCF_d','completo', 'CH06','IX_TOT','casadpto','basural','leer','NIVEL_ED','mujer',
                                'casadounido','ESTADO','CAT_INAC','CAT_OCUP','otros_ing_nolab','informal']].columns
allvars

Index(['logIPCF_d', 'completo', 'CH06', 'IX_TOT', 'casadpto', 'basural',
       'leer', 'NIVEL_ED', 'mujer', 'casadounido', 'ESTADO', 'CAT_INAC',
       'CAT_OCUP', 'otros_ing_nolab', 'informal'],
      dtype='object')

In [50]:
hp_model = sklearn_sm(sm.GLM,
                      MS(allvars))

cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0) # use same splits for each degree

X = individual_NEA[['logIPCF_d','completo', 'CH06','IX_TOT','casadpto','basural','leer','NIVEL_ED','mujer',
                                'casadounido','ESTADO','CAT_INAC','CAT_OCUP','otros_ing_nolab','informal']], 
Y = individual_NEA['completo']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=cv)
cv_err = np.mean(cv_results['test_score'])
cv_err

ValueError: Found input variables with inconsistent numbers of samples: [1, 34860]